In [ ]:
import pandas as pd
import json

from adodbapi.examples import db_table_names
from debugpy._vendored.pydevd.stubs._django_manager_body import db
from tqdm import tqdm

In [ ]:
# load multi mapped works into a dataframe

mm_csv_path = "E:/mapping_oct_23/multi_map/multi_mapped_omids.csv"

df = pd.read_csv(mm_csv_path)

In [ ]:
df

In [ ]:
def read_full_metadata(jsonl_file):
    with open(jsonl_file) as f:
        for line in f:
            data = json.loads(line)
            yield data

In [ ]:
from pprint import pprint

for ent in tqdm(read_full_metadata("E:/multi_mapped_full_metadata/works.json")):
    # oaid = ent['id'].removeprefix('https://openalex.org/')
    pprint(ent)
    break

    

In [7]:
import psycopg2 as pg

dbname = 'mm_openalex'
postgres_user = 'postgres'
postgres_password = input(f'Insert password for user {postgres_user}')
# connect to the database
conn = pg.connect(dbname=dbname, host='localhost', user= postgres_user, password = postgres_password, port='5432')
cur = conn.cursor()
query = "SELECT w.work_id FROM openalex.works_ids w WHERE w.pmid = %s;"
cur.execute(query, ('https://pubmed.ncbi.nlm.nih.gov/21793740',))
rows = cur.fetchall()
for row in rows:
    print(row)
cur.close()
conn.close()

('https://openalex.org/W2157612734',)


In [48]:
# # from pprint import pprint
# import pandas as pd
# from collections import defaultdict
# 
# 
# mm_works_json_path = "E:/multi_mapped_full_metadata/works.json"
# 
# 
# df = pd.DataFrame()
# for ent in tqdm(read_full_metadata(mm_works_json_path)):
#     out = {}
#     out['id'] = ent['id']
#     out['title'] = ent['title']
#     out['doi'] = ent['doi']
#     out['pmid'] = ent['ids'].get('pmid')
#     out['pmcid'] = ent['ids'].get('pmcid')
#     out['mag'] = ent['ids'].get('mag')
#     out['type'] = ent['type']
#     out['type_crossref'] = ent['type_crossref']
#     if ent.get('primary_location'):
#         if ent['primary_location'].get('source'):
#             out['primary_location_id'] = ent['primary_location']['source'].get('id')
#             out['primary_location_type'] = ent['primary_location']['source'].get('type')
#         else:
#             out['primary_location_id'] = None
#             out['primary_location_type'] = None
#     else:
#         out['primary_location_id'] = None
#         out['primary_location_type'] = None
# 
#     out['locations']:list = ent['locations']
# 
#     
#     row_df = pd.json_normalize(out)
#     
#     df = pd.concat([df, row_df], ignore_index=True)
# 
# 
# df
#     
#     
#     
#     

347717it [6:38:51, 14.53it/s]


,id,title,doi,pmid,pmcid,mag,type,type_crossref,primary_location_id,primary_location_type,locations
0,https://openalex.org/W2093406744,The processing and representation of fractions...,https://doi.org/10.1016/j.neuroimage.2009.03.041,https://pubmed.ncbi.nlm.nih.gov/19328235,None,2093406744,article,journal-article,https://openalex.org/S4306525036,repository,"[{'license': None, 'pdf_url': None, 'is_oa': F..."
1,https://openalex.org/W2074493785,Are gram-negative bacteria a contraindication ...,https://doi.org/10.1067/mva.1992.38625,https://pubmed.ncbi.nlm.nih.gov/1522635,None,2074493785,article,journal-article,https://openalex.org/S4306525036,repository,"[{'license': None, 'pdf_url': None, 'is_oa': F..."
2,https://openalex.org/W2147314816,Automated diffraction tomography for the struc...,https://doi.org/10.1002/anie.201204963,https://pubmed.ncbi.nlm.nih.gov/22976879,None,2147314816,article,journal-article,https://openalex.org/S4306525036,repository,"[{'license': None, 'pdf_url': None, 'is_oa': F..."
3,https://openalex.org/W2016627268,Percutaneous transluminal renal angioplasty ve...,https://doi.org/10.1067/mva.1993.45062,https://pubmed.ncbi.nlm.nih.gov/8230572,None,2016627268,article,journal-article,https://openalex.org/S4306525036,repository,"[{'license': None, 'pdf_url': None, 'is_oa': F..."
4,https://openalex.org/W2161822684,Simple physical model of collagen fibrillogene...,https://doi.org/10.1006/jmbi.1994.0182,https://pubmed.ncbi.nlm.nih.gov/7723033,None,2161822684,article,journal-article,https://openalex.org/S4306525036,repository,"[{'license': None, 'pdf_url': None, 'is_oa': F..."
...,...,...,...,...,...,...,...,...,...,...,...
347712,https://openalex.org/W4306253068,"Observation of <mml:math xmlns:mml=""http://www...",https://doi.org/10.1103/physrevd.106.072004,None,None,None,article,journal-article,https://openalex.org/S4210238307,journal,"[{'license': 'cc-by', 'is_accepted': True, 'is..."
347713,https://openalex.org/W4306254261,Quantifying groundwater phosphorus flux to riv...,https://doi.org/10.1007/s11356-022-23574-9,https://pubmed.ncbi.nlm.nih.gov/36242662,None,None,article,journal-article,https://openalex.org/S87561820,journal,"[{'license': None, 'is_accepted': False, 'is_p..."
347714,https://openalex.org/W4306254484,Comparison of hypoxia among four river-dominat...,https://doi.org/10.17615/h920-vv92,None,None,None,article,journal-article,https://openalex.org/S4306401075,repository,"[{'license': None, 'is_accepted': False, 'is_p..."
347715,https://openalex.org/W4306409695,Effect of micromotion and local stress in quan...,https://doi.org/10.1103/physreva.106.042612,None,None,None,article,journal-article,https://openalex.org/S4210182288,journal,"[{'license': None, 'is_accepted': False, 'is_p..."
